In [33]:
print("Notebook działa")


Notebook działa


# TaskFlow – konsolowy menedżer zadań w Pythonie

## Opis projektu
TaskFlow to prosta aplikacja konsolowa napisana w języku Python, służąca do zarządzania zadaniami (To-Do List).
Użytkownik może dodawać zadania, wyświetlać ich listę, oznaczać je jako ukończone, usuwać je oraz sortować według priorytetu lub terminu wykonania.

Dane są zapisywane w lokalnej bazie danych SQLite, co umożliwia ich trwałe przechowywanie pomiędzy uruchomieniami programu.

## Cel projektu
Celem projektu jest praktyczne zastosowanie zagadnień omawianych na zajęciach z warsztatów języków programowania.


In [34]:
import sqlite3
import json
from dataclasses import dataclass
from datetime import datetime
from typing import Optional, List, Any


In [35]:
class TaskFlowError(Exception):
    """Bazowy wyjątek aplikacji."""
    pass
class TaskNotFoundError(TaskFlowError):
    """Wyjątek, gdy nie znaleziono zadania."""
    pass
class ValidationError(TaskFlowError):
    """Wyjątek dla niepoprawnych danych."""
    pass


In [36]:
@dataclass
class Task:
    id: Optional[int]
    title: str
    priority: int              # 1-5
    due_date: Optional[str]    # "YYYY-MM-DD" albo None
    done: bool = False

    def to_dict(self) -> dict:
        return {
            "id": self.id,
            "title": self.title,
            "priority": self.priority,
            "due_date": self.due_date,
            "done": self.done
        }

    @staticmethod
    def from_dict(d: dict) -> "Task":
        return Task(
            id=d.get("id"),
            title=str(d.get("title", "")).strip(),
            priority=int(d.get("priority", 3)),
            due_date=d.get("due_date"),
            done=bool(d.get("done", False))
        )


In [37]:
t1 = Task(id=None, title="Zrobić projekt", priority=5, due_date="2026-01-30")
print(t1)


Task(id=None, title='Zrobić projekt', priority=5, due_date='2026-01-30', done=False)


In [38]:
t1 = Task(id=1, title="Test JSON", priority=4, due_date="2026-01-30", done=False)

d = t1.to_dict()
print("Słownik:", d)

t2 = Task.from_dict(d)
print("Obiekt z dict:", t2)


Słownik: {'id': 1, 'title': 'Test JSON', 'priority': 4, 'due_date': '2026-01-30', 'done': False}
Obiekt z dict: Task(id=1, title='Test JSON', priority=4, due_date='2026-01-30', done=False)


In [39]:
class TaskRepository:
    def __init__(self, db_path: str = "taskflow.db") -> None:
        self.db_path = db_path
        self._init_db()

    def _connect(self) -> sqlite3.Connection:
        return sqlite3.connect(self.db_path)

    def _init_db(self) -> None:
        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute("""
                CREATE TABLE IF NOT EXISTS tasks (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT NOT NULL,
                    priority INTEGER NOT NULL,
                    due_date TEXT,
                    done INTEGER NOT NULL
                )
            """)
            conn.commit()
    def add(self, task: Task) -> int:
        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute(
                "INSERT INTO tasks (title, priority, due_date, done) VALUES (?, ?, ?, ?)",
                (task.title, task.priority, task.due_date, int(task.done))
            )
            conn.commit()
            return int(cur.lastrowid)

    def get_all(self) -> List[Task]:
        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute("SELECT id, title, priority, due_date, done FROM tasks")
            rows = cur.fetchall()

        return [
            Task(id=r[0], title=r[1], priority=r[2], due_date=r[3], done=bool(r[4]))
            for r in rows
        ]
    def get_by_id(self, task_id: int) -> Task:
        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute(
                "SELECT id, title, priority, due_date, done FROM tasks WHERE id = ?",
                (task_id,)
            )
            row = cur.fetchone()

        if row is None:
            raise TaskNotFoundError(f"Nie znaleziono zadania o ID={task_id}")

        return Task(
            id=row[0],
            title=row[1],
            priority=row[2],
            due_date=row[3],
            done=bool(row[4])
        )

    def mark_done(self, task_id: int, done: bool = True) -> None:
        # sprawdzenie czy istnieje
        _ = self.get_by_id(task_id)

        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute(
                "UPDATE tasks SET done = ? WHERE id = ?",
                (int(done), task_id)
            )
            conn.commit()

    def delete(self, task_id: int) -> None:
        # sprawdzenie czy istnieje
        _ = self.get_by_id(task_id)

        with self._connect() as conn:
            cur = conn.cursor()
            cur.execute(
                "DELETE FROM tasks WHERE id = ?",
                (task_id,)
            )
            conn.commit()


In [40]:
repo = TaskRepository()
print("Utworzono repo. Plik bazy:", repo.db_path)


Utworzono repo. Plik bazy: taskflow.db


In [41]:
with sqlite3.connect("taskflow.db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print("Tabele w bazie:", cur.fetchall())


Tabele w bazie: [('tasks',), ('sqlite_sequence',)]


In [42]:
repo = TaskRepository()

task = Task(id=None, title="Pierwsze zadanie w bazie", priority=3, due_date="2026-01-30", done=False)
new_id = repo.add(task)

print("Dodano zadanie. ID =", new_id)


Dodano zadanie. ID = 5


In [43]:
tasks = repo.get_all()
print("Liczba zadań:", len(tasks))
for t in tasks:
    print(t)


Liczba zadań: 4
Task(id=2, title='Niski priorytet', priority=1, due_date=None, done=False)
Task(id=3, title='Wysoki priorytet', priority=5, due_date=None, done=False)
Task(id=4, title='Średni priorytet', priority=3, due_date=None, done=False)
Task(id=5, title='Pierwsze zadanie w bazie', priority=3, due_date='2026-01-30', done=False)


In [44]:
repo = TaskRepository()
print("Nowe repo OK")


Nowe repo OK


In [46]:
try:
    task = repo.get_by_id(1)
    print(task)
except TaskNotFoundError as e:
    print("OK – wyjątek złapany:", e)



OK – wyjątek złapany: Nie znaleziono zadania o ID=1


In [ ]:
task = repo.get_by_id(new_id)
print(task)


In [ ]:
repo.mark_done(task.id, True)
print("Po mark_done:", repo.get_by_id(task.id))


In [ ]:
repo.get_by_id(999999)


In [ ]:
repo.delete(1)
print("Usunięto zadanie ID=1")


In [ ]:
print(repo.get_all())


In [ ]:
repo.get_by_id(1)


In [ ]:
def merge_sort(tasks: List[Task], key=lambda t: t.priority) -> List[Task]:
    """
    Merge Sort: sortowanie przez scalanie.
    Złożoność: O(n log n) w każdym przypadku.
    """
    if len(tasks) <= 1:
        return tasks

    mid = len(tasks) // 2
    left = merge_sort(tasks[:mid], key=key)
    right = merge_sort(tasks[mid:], key=key)

    return _merge(left, right, key)


def _merge(left: List[Task], right: List[Task], key) -> List[Task]:
    result = []
    i = j = 0

    while i < len(left) and j < len(right):
        if key(left[i]) <= key(right[j]):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # dopisz resztę
    result.extend(left[i:])
    result.extend(right[j:])
    return result


In [ ]:
repo = TaskRepository()

# dodajmy kilka testowych zadań
repo.add(Task(id=None, title="Niski priorytet", priority=1, due_date=None, done=False))
repo.add(Task(id=None, title="Wysoki priorytet", priority=5, due_date=None, done=False))
repo.add(Task(id=None, title="Średni priorytet", priority=3, due_date=None, done=False))

tasks = repo.get_all()

sorted_tasks = merge_sort(tasks, key=lambda t: t.priority)

print("=== PRZED ===")
for t in tasks:
    print(t.id, t.title, t.priority)

print("\n=== PO (Merge Sort po priority rosnąco) ===")
for t in sorted_tasks:
    print(t.id, t.title, t.priority)


In [ ]:
repo = TaskRepository()

# 1) usuwamy zadanie o ID=1 (jeśli istnieje)
try:
    repo.delete(1)
    print("✅ Usunięto zadanie ID=1")
except TaskNotFoundError:
    print("ℹ️ Zadanie ID=1 nie istnieje (już było usunięte)")


In [ ]:
print("Aktualne zadania w bazie:")
for t in repo.get_all():
    print(t.id, t.title, t.priority, t.done)


In [ ]:
repo.get_by_id(1)
